# algoritmo_deidentificacion_regex

Este script permite deidentificar texto libre proveniente de registros clínicos electrónicos.

0. Configura el ambiente.
1. Carga los datos (input)
~~Chequea la sanidad de los datos (backlog)~~
2. Preprocesa los textos ("limpieza")
~~Parametriza el modelo de deidentificación a aplicar (cambaa, regex, regex+spacy)~~
~~Chequea la cantidad de textos y de ser necesario, particiona la tarea.~~
3. Parametriza el output deseado (texto censurado, resaltado, reemplazado, clasificado). 
4. Procesa los textos (aplica las reglas).
5. Devuelve los datos procesados (output)


## Lista de entidades

- PACIENTE: PERSONA
- EDAD: 
- GÉNERO: no se detecta. 
- FAMILIAR: PERSONA
- ~~DRX~~
- ~~FECHA~~
- ~~EFECTOR~~
- ~~INSTITUCIÓN~~: escolares.
- ~~DIRECCIÓN~~
- ~~ZONA~~
- ~~PAÍS~~
- ~~NÚM_TELÉFONO~~
- ~~CORREO_ELECTRÓNICO~~
- ~~NÚM_DNI~~
- ~~NÚM_CUIT_CUIL~~
- ~~PASAPORTE~~
- ~~MATRICULA~~
- ~~EPOF:~~ queda pendiente estrategia para reemplazar por el nombre de la EPof entre <>
- ~~PATENTE~~: no se cuenta con ejemplos (no se detectaron en la base de datos).
- NÚM_SERIE_DISPOSITIVOS: no se detectan.
- ~~OTROS_NÚM~~: no se detectan.
- DUDOSOS: no se detectan.

## 0. Configura el ambiente

In [1]:
import pandas as pd
import os
import re

# This code allows to view full lenght text
pd.set_option("display.max_colwidth", None)

In [2]:
def remove_accents(text: str)  -> str:
    """Removes common accent characters, converts to lowercase."""
    new = re.sub(r'[àáâãäå]', 'a', text)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    new = re.sub(r'[ÀÁÂÃÄÅ]', 'A', new)
    new = re.sub(r'[ÈÉÊË]', 'E', new)
    new = re.sub(r'[ÌÍÎÏ]', 'I', new)
    new = re.sub(r'[ÒÓÔÕÖ]', 'O', new)
    new = re.sub(r'[ÙÚÛÜ]', 'U', new)
    return new

## 1. Carga los datos (input)

In [3]:
INPUT_DIR = '/home/usuario/Documentos/TrabajoEspecial/Datasets/SPGExt/integral/'
TEXT_COLUMN = 'informes'

In [4]:
# Obtener lista de archivos .txt
txt_files = sorted([f for f in os.listdir(INPUT_DIR) if f.endswith(".txt")])

# Mostrar los archivos encontrados
print(txt_files[:5])

data = []

for txt_file in txt_files:
    txt_path = os.path.join(INPUT_DIR, txt_file)
    
    with open(txt_path, 'r', encoding='utf-8') as f:
        texto = f.read()
    
    data.append({
        TEXT_COLUMN: texto
    })

documents = pd.DataFrame(data)

# Mostrar primeras filas
print(documents.head(2))

['000096468.txt', '002769099.txt', '003666606.txt', '004954246.txt', '005766636.txt']
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

## 2. Preprocesa los textos ("limpieza")

In [5]:
# Genera una nueva columna con el texto preprocesado
documents.rename(columns={TEXT_COLUMN: 'original'}, inplace=True)
documents['preprocessed'] = documents['original'].apply(remove_accents)
# Apply further transformations if necessary

# Remove nan
documents = documents.dropna()
documents.head(2)

,original,preprocessed
0,"Datos del paciente.\nNombre: Emilia García López\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGénero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCódigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTeléfono fijo: +34 91 123 45 67\nTeléfono móvil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondición de riesgo: Persona con discapacidad\nDatos asistenciales.\nMédico: Dra. Sofía Pérez García. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clínico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con síntomas de dolor torácico radiante al brazo izquierdo y dificultad para respirar. La exploración física revela taquicardia, hipertensión y roncus bilaterales. Los análisis de sangre muestran elevación de la creatinina y la troponina. Se solicitan estudios de imagen (Rx tórax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventrículo izquierdo. Se instaura tratamiento con anticoagulantes y analgésicos, y se realiza una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertensión\n- Diabetes mellitus tipo 2\nEvolución y comentarios:\nEl paciente se encuentra en una evolución estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado análisis de sangre y estudios de imagen para confirmar el diagnóstico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgésicos. Se le realizará una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada.\nRemitido por:\nSe reenvía a la unidad de cardiología para seguir el tratamiento y la evolución del paciente.\n","Datos del paciente.\nNombre: Emilia Garcia Lopez\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGenero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCodigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTelefono fijo: +34 91 123 45 67\nTelefono movil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondicion de riesgo: Persona con discapacidad\nDatos asistenciales.\nMedico: Dra. Sofia Perez Garcia. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clinico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con sintomas de dolor toracico radiante al brazo izquierdo y dificultad para respirar. La exploracion fisica revela taquicardia, hipertension y roncus bilaterales. Los analisis de sangre muestran elevacion de la creatinina y la troponina. Se solicitan estudios de imagen (Rx torax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventriculo izquierdo. Se instaura tratamiento con anticoagulantes y analgesicos, y se realiza una angioplastia coronaria para restaurar la circulacion sanguinea en el area afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertension\n- Diabetes mellitus tipo 2\nEvolucion y comentarios:\nEl paciente se encuentra en una evolucion estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado analisis de sangre y estudios de imagen para confirmar el diagnostico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgesicos. Se le realizara una angioplastia coronaria para restaurar 

## 3. Parametriza el output deseado 

Ejemplos de outputs posibles:

1. Texto resaltado: La usuaria <María>
2. Texto censurado: La usuaria xxxx
3. Texto clasificado: La usuaria PACIENTE

In [6]:
from enum import Enum

class AnonymizationMode(Enum):
    HIGHLIGHT = 1
    CENSOR = 2
    REPLACE = 3

# Ingresar el tipo de output deseado
OUTPUT_TYPE = AnonymizationMode.REPLACE

ILLEGAL_CHAR_REGEX = r"[\/<>\:\"\\|\?\*\+\[\]\(\)]"

In [7]:
class RegexAnnonimizer(object):
    """
    The RegexAnonymizer class is designed to find and replace matches of a given regular expression
    within a text string, according to a specified output type. The purpose of this class is to 
    anonymize text by either highlighting, censoring, or replacing specific patterns in the text.

    Attributes:
        regex (str): The regular expression pattern used to identify the text to be anonymized.
        output_type (AnonymizationMode): The mode of anonymization, determining the action taken on matches.
        category (str): The text used for replacement when using the REPLACE output type.
        verbose (bool): Whether to print the replaced text. Useful for debug.

    Methods:
        anonymize(text: str) -> str:
            Anonymizes the input text based on the provided regular expression and output type.
            Depending on the output type, it either highlights, censors, or replaces the matched text.

    Output Types:
        1 (AnonymizationMode.HIGHLIGHT): Wraps the matched text with angle brackets.
            Example: "The user Maria" -> "The user <Maria>"
        2 (AnonymizationMode.CENSOR): Replaces the matched text with 'x' characters, maintaining the length of the original text.
            Example: "The user Maria" -> "The user xxxxx"
        3 (AnonymizationMode.REPLACE): Replaces the matched text with the provided replacement text.
            Example: "The user Maria" -> "The user PACIENTE"
    """
    def __init__(self,
                 content_regex: str,
                 output_type: AnonymizationMode = AnonymizationMode.REPLACE,
                 category: str = None,
                 re_flags: re.RegexFlag = None,
                 verbose: bool = False):
        """
        Initializes the RegexAnonymizer instance with the specified parameters.
        
        Args:
            content_regex (str): The regular expression pattern used to identify the text for anonymization.
            output_type (AnonymizationMode, optional): The mode of anonymization, determining the action taken on matches.
                Default is AnonymizationMode.REPLACE.
            category (str, optional): The text used for replacement when using the REPLACE output type. Required
                if output_type is AnonymizationMode.REPLACE.
            re_flags (re.RegexFlag, optional): flags to pass to re.compile when compiling patterns. Default is None.
            verbose (bool, optional): If True, enables verbose output, providing details about the anonymization
                process. Default is False.
        
        Raises:
            ValueError: If output_type is not an instance of AnonymizationMode.
            ValueError: If output_type is AnonymizationMode.REPLACE and category is not provided.
        """
        if not isinstance(output_type, AnonymizationMode):
            raise ValueError('`output_type` must be an instance of `AnonymizationMode`.')
        self.output_type = output_type
        if re_flags is None:
            self.content_regex = re.compile(content_regex)
        else:
            self.content_regex = re.compile(content_regex, re_flags)
        if self.output_type == AnonymizationMode.REPLACE and not category:
            raise ValueError('When `output_type` is `AnonymizationMode.REPLACE`, you need to provide '
                             'a valid string value for `category`.')
        self.category = category
        self.verbose = verbose

    def _replace_match(self, match: re.Match) -> str:
        """Replaces all matches in string according to self.output_type."""
        matched_name = match.group(0)
        if self.verbose:
            # TODO replace with logger
            print('Replacing', matched_name)

        if self.output_type == AnonymizationMode.HIGHLIGHT:
            return f"<{matched_name}>"
        if self.output_type == AnonymizationMode.CENSOR:
            return "x" * len(matched_name)
        if self.output_type == AnonymizationMode.REPLACE:
            return self.category
        raise ValueError('Output type not suported', self.output_type)

    def _post_process(self, anonymized_text: str) -> str:
        """Collapse consecutive replacements into a single occurrence of category."""
        if self.output_type == AnonymizationMode.REPLACE:
            anonymized_text = re.sub(r'({})(\s+\1)+'.format(self.category), r'\1', anonymized_text)
        return anonymized_text

    def anonymize(self, text: str) -> str:
        """Anonymizes text by replacing content matches, according to output type."""
        # Replace the matched names with the desired output
        anonymized_text = self.content_regex.sub(self._replace_match, text)
        return self._post_process(anonymized_text)

class PrefixRegexAnnonimizer(RegexAnnonimizer):
    """
    A specialized anonymizer that detects and replaces text based on a prefix and content pattern.
    
    This class inherits from RegexAnonymizer and extends its functionality by allowing two regular
    expressions to be specified: one for the prefix and another for the content. If the content is 
    found immediately following the prefix, it is subject to anonymization according to the specified 
    output type. This is particularly useful for identifying and anonymizing structured information, 
    
    such as phone numbers, addresses, or any other context-specific data that is preceded by a 
    particular keyword or phrase.
    
    Attributes:
        prefix_regex (str): A regular expression pattern to identify the prefix text.
        content_regex (str): A regular expression pattern to identify the content to be anonymized.
        output_type (AnonymizationMode): Specifies how the matched content should be anonymized.
        category (str): The replacement text used when the output_type is AnonymizationMode.REPLACE.
        verbose (bool): Determines if detailed information about the anonymization process should be printed.
    
    Methods:
        anonymize(text: str) -> str:
            Processes the input text and replaces occurrences of content following the specified 
            prefix according to the anonymization rules defined by the output_type.
    """
    def __init__(self,
                 prefix_regex: str,
                 content_regex: str,
                 output_type: AnonymizationMode = AnonymizationMode.REPLACE,
                 category: str = None,
                 re_flags: re.RegexFlag = None,
                 verbose: bool = False):
        """
        Initializes the PrefixRegexAnnonimizer instance with the specified parameters.
        
        Args:
            prefix_regex: (str): The regular expression pattern used to identify the prefix before the text to anonymize.
            content_regex: (str): The regular expression pattern used to identify the text for anonymization.
            output_type (AnonymizationMode, optional): The mode of anonymization, determining the action taken on matches.
                Default is AnonymizationMode.REPLACE.
            category (str, optional): The text used for replacement when using the REPLACE output type. Required
                if output_type is AnonymizationMode.REPLACE.
            re_flags (re.RegexFlag, optional): flags to pass to re.compile when compiling patterns. Default is None.
            verbose (bool, optional): If True, enables verbose output, providing details about the anonymization
                process. Default is False.
        
        Raises:
            ValueError: If output_type is not an instance of AnonymizationMode.
            ValueError: If output_type is AnonymizationMode.REPLACE and category is not provided.
        """
        super().__init__(
            content_regex=content_regex,
            output_type=output_type,
            category=category,
            verbose=verbose,
            re_flags=re_flags
        )
        if re_flags is None:
            self.prefix_regex = re.compile(prefix_regex)
            self._full_regex = re.compile(fr'(?P<prefix>{prefix_regex})\s*(?P<content>{content_regex})')
        else:
            self.prefix_regex = re.compile(prefix_regex, re_flags)
            self._full_regex = re.compile(fr'(?P<prefix>{prefix_regex})\s*(?P<content>{content_regex})', re_flags)
    
    def _replace_if_prefix(self, match: re.Match):
        """Replace matches to self. of regex only if matches self._full_regex.""" 
        # Check if a phone-related term precedes the number, and then replace
        return match.group('prefix') + ' ' + self.content_regex.sub(self._replace_match, match.group('content'), count=1)

    def anonymize(self, text: str) -> str:
        """Anonymizes text by replacing content matches that follow prefix matches, according to output type."""
        # Replace the matched names with the desired output
        anonymized_text = self._full_regex.sub(self._replace_if_prefix, text)
        return self._post_process(anonymized_text)

## 4. Procesa los textos (aplica las reglas).

### FECHA

In [8]:
import datetime
## Expresiones regulares para FECHA
regexp_day = r'(\d{1,2}[\/ |-](\d{1,2}|Enero|Ene|Febrero|Feb|Marzo|Mar|Abril|Abr|Mayo|May|Junio|Jun|Julio|Jul|Agosto|Ago|Septiembre|Sep|Octubre|Oct|Noviembre|Nov|Diciembre|Dic)[\/ |-]\d{2,4})'
regexp_month = r'\b(Enero|Ene|Febrero|Feb|Marzo|Mar|Abril|Abr|Mayo|May|Junio|Jun|Julio|Jul|Agosto|Ago|Septiembre|Sep|Octubre|Oct|Noviembre|Nov|Diciembre|Dic)\b'
regexp_year = r'\b(19[89][0-9]|20[0-4][0-9]|{})\b'.format(datetime.date.today().year)
regexp_date = '|'.join([regexp_day, regexp_month, regexp_year])

date_anonymizer = RegexAnnonimizer(
    content_regex=regexp_date,
    category='XFECHAX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE
)
# Ejemplos:
print(date_anonymizer.anonymize("Se programa turno para dìa 21/03/2021. Jueves 17:00hrs, 31 de Marzo."))
print(date_anonymizer.anonymize("Se programa turno para octubre. Control 2023"))

Se programa turno para dìa XFECHAX. Jueves 17:00hrs, 31 de XFECHAX.
Se programa turno para XFECHAX. Control XFECHAX


In [9]:
documents['anonymized'] = documents['preprocessed'].apply(date_anonymizer.anonymize)

In [10]:
documents[documents['anonymized'].str.contains('XFECHAX', case=False)].head(2)

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: Emilia García López\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGénero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCódigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTeléfono fijo: +34 91 123 45 67\nTeléfono móvil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondición de riesgo: Persona con discapacidad\nDatos asistenciales.\nMédico: Dra. Sofía Pérez García. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clínico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con síntomas de dolor torácico radiante al brazo izquierdo y dificultad para respirar. La exploración física revela taquicardia, hipertensión y roncus bilaterales. Los análisis de sangre muestran elevación de la creatinina y la troponina. Se solicitan estudios de imagen (Rx tórax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventrículo izquierdo. Se instaura tratamiento con anticoagulantes y analgésicos, y se realiza una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertensión\n- Diabetes mellitus tipo 2\nEvolución y comentarios:\nEl paciente se encuentra en una evolución estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado análisis de sangre y estudios de imagen para confirmar el diagnóstico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgésicos. Se le realizará una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada.\nRemitido por:\nSe reenvía a la unidad de cardiología para seguir el tratamiento y la evolución del paciente.\n","Datos del paciente.\nNombre: Emilia Garcia Lopez\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGenero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCodigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTelefono fijo: +34 91 123 45 67\nTelefono movil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondicion de riesgo: Persona con discapacidad\nDatos asistenciales.\nMedico: Dra. Sofia Perez Garcia. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clinico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con sintomas de dolor toracico radiante al brazo izquierdo y dificultad para respirar. La exploracion fisica revela taquicardia, hipertension y roncus bilaterales. Los analisis de sangre muestran elevacion de la creatinina y la troponina. Se solicitan estudios de imagen (Rx torax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventriculo izquierdo. Se instaura tratamiento con anticoagulantes y analgesicos, y se realiza una angioplastia coronaria para restaurar la circulacion sanguinea en el area afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertension\n- Diabetes mellitus tipo 2\nEvolucion y comentarios:\nEl paciente se encuentra en una evolucion estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado analisis de sangre y estudios de imagen para confirmar el diagnostico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgesicos. Se le realizara una angioplastia coronaria para

### NUM_TELEFONO

In [11]:
## Expresión regular para NUM_TELEFONO
regexp_phone_prefix =  '(tel[ée]fono|(tel(?![aA-zZ]))|celular|(cel(?![aA-zZ])))\s*(?:n\s*[\.\°]|.)?\s*[:\.]?'
regexp_phone_number = '[0-9]+'
separators = r'\s*[:\.]?\s*'

phone_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_phone_prefix,
    content_regex=regexp_phone_number,
    re_flags=re.IGNORECASE,
    category='XTELEFONOX',
    output_type=OUTPUT_TYPE,
)

print(phone_anonymizer.anonymize("Paciente indica llamar a tel 3514790943 sin poder comunicarse."))
print(phone_anonymizer.anonymize("Paciente indica llamar a telefono n° 3514790943 sin poder comunicarse."))
print(phone_anonymizer.anonymize("Rescinde celular durante 400 dias"))
print(phone_anonymizer.anonymize("Proximo turno enero, mandar recordatorio cel: 3514790943"))
print(phone_anonymizer.anonymize("Proximo turno enero, mandar recordatorio cel.: 3514790943"))

Paciente indica llamar a tel 3 XTELEFONOX sin poder comunicarse.
Paciente indica llamar a telefono n°  XTELEFONOX sin poder comunicarse.
Rescinde celular durante 400 dias
Proximo turno enero, mandar recordatorio cel:  XTELEFONOX
Proximo turno enero, mandar recordatorio cel.: XTELEFONOX


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_71236/1487379825.py:2: SyntaxWarning: invalid escape sequence '\s'
  regexp_phone_prefix =  '(tel[ée]fono|(tel(?![aA-zZ]))|celular|(cel(?![aA-zZ])))\s*(?:n\s*[\.\°]|.)?\s*[:\.]?'


In [12]:
documents['anonymized'] = documents.anonymized.apply(phone_anonymizer.anonymize)

In [13]:
documents[documents['anonymized'].str.contains('XNUM_TELEFONOX',case=False)].head(2)

,original,preprocessed,anonymized


### CORREO ELECTRÓNICO

In [14]:
## Expresión regular para CORREO ELECTRÓNICO
regexp_email = r'[\w\.-]+@[\w\.-]+\.\w+'

email_anonymizer = RegexAnnonimizer(
    content_regex=regexp_email,
    category='XCORREO_ELECTRONICOX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(email_anonymizer.anonymize("Contactar paciente a fakemail@fakedomain.com"))
print(email_anonymizer.anonymize("Contactar paciente a fakemail@fakedomain.com.ar"))
print(email_anonymizer.anonymize("Paciente refiere conflicto con herman@"))

Contactar paciente a XCORREO_ELECTRONICOX
Contactar paciente a XCORREO_ELECTRONICOX
Paciente refiere conflicto con herman@


In [15]:
documents['anonymized'] = documents['anonymized'].apply(email_anonymizer.anonymize)

In [16]:
documents[documents['anonymized'].str.contains('XCORREO_ELECTRONICOX',case=False)]

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: Emilia García López\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGénero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCódigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTeléfono fijo: +34 91 123 45 67\nTeléfono móvil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondición de riesgo: Persona con discapacidad\nDatos asistenciales.\nMédico: Dra. Sofía Pérez García. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clínico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con síntomas de dolor torácico radiante al brazo izquierdo y dificultad para respirar. La exploración física revela taquicardia, hipertensión y roncus bilaterales. Los análisis de sangre muestran elevación de la creatinina y la troponina. Se solicitan estudios de imagen (Rx tórax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventrículo izquierdo. Se instaura tratamiento con anticoagulantes y analgésicos, y se realiza una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertensión\n- Diabetes mellitus tipo 2\nEvolución y comentarios:\nEl paciente se encuentra en una evolución estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado análisis de sangre y estudios de imagen para confirmar el diagnóstico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgésicos. Se le realizará una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada.\nRemitido por:\nSe reenvía a la unidad de cardiología para seguir el tratamiento y la evolución del paciente.\n","Datos del paciente.\nNombre: Emilia Garcia Lopez\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGenero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCodigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTelefono fijo: +34 91 123 45 67\nTelefono movil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondicion de riesgo: Persona con discapacidad\nDatos asistenciales.\nMedico: Dra. Sofia Perez Garcia. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clinico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con sintomas de dolor toracico radiante al brazo izquierdo y dificultad para respirar. La exploracion fisica revela taquicardia, hipertension y roncus bilaterales. Los analisis de sangre muestran elevacion de la creatinina y la troponina. Se solicitan estudios de imagen (Rx torax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventriculo izquierdo. Se instaura tratamiento con anticoagulantes y analgesicos, y se realiza una angioplastia coronaria para restaurar la circulacion sanguinea en el area afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertension\n- Diabetes mellitus tipo 2\nEvolucion y comentarios:\nEl paciente se encuentra en una evolucion estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado analisis de sangre y estudios de imagen para confirmar el diagnostico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgesicos. Se le realizara una angioplastia coronaria para

### NUM_DNI

In [17]:
## Expresión regular para NUM_DNI

regexp_dni_prefix = r'(d\.n\.i|dni|doc(?= )|documento)(\sn\s?\.?\°?)?\s*[:\.]?\s*'
regexp_dni_numbers = r'\b\d{1,3}(?:\.*\d{3})*\b'

dni_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_dni_prefix,
    content_regex=regexp_dni_numbers,
    re_flags=re.IGNORECASE,
    category='XNUM_DNIX',
    output_type=OUTPUT_TYPE,
)

print(dni_anonymizer.anonymize("Paciente dni 27.384.234 control prenatal semana 27"))
print(dni_anonymizer.anonymize("Paciente dni n° 27.384.234 control prenatal semana 27"))
print(dni_anonymizer.anonymize("Paciente dni n ° 27.384.234 control prenatal semana 27"))
print(dni_anonymizer.anonymize("Paciente dni 27384234, control prenatal"))
print(dni_anonymizer.anonymize("Paciente dni: 27384234 control prenatal"))
print(dni_anonymizer.anonymize("Para mas detalles ver documento 25"))
print(dni_anonymizer.anonymize("Refiere extravio dni sin sintomatologia tel 27384234"))

Paciente dni  XNUM_DNIX control prenatal semana 27
Paciente dni n°  XNUM_DNIX control prenatal semana 27
Paciente dni n °  XNUM_DNIX control prenatal semana 27
Paciente dni  XNUM_DNIX, control prenatal
Paciente dni:  XNUM_DNIX control prenatal
Para mas detalles ver documento  XNUM_DNIX
Refiere extravio dni sin sintomatologia tel 27384234


In [18]:
documents['anonymized'] = documents['anonymized'].apply(dni_anonymizer.anonymize)

In [19]:
documents[documents['anonymized'].str.contains('XNUM_DNIX',case=False)]

,original,preprocessed,anonymized


### NUM_CUIT_CUIL

In [20]:
## Expresión regular para NUM_CUIT_CUIL
regexp_cuil_prefix = r'(c.u.i.l.|cuil|c.u.i.t|cuit)(\sn\s?\.?\°?)?\s*[:\.]?\s*'
# Numbers, or numbers with . or -
regexp_cuil_numbers = r'\b(?:\d(?:[\.\-]\d)*)+\b'

cuit_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_cuil_prefix,
    content_regex=regexp_cuil_numbers,
    re_flags=re.IGNORECASE,
    category='XNUM_CUIT_CUILX',
    output_type=OUTPUT_TYPE,
)

print(cuit_anonymizer.anonymize("Paciente CUIT 25-27.384.234-4 control prenatal semana 27"))
print(cuit_anonymizer.anonymize("Paciente cuil 25-27384234-4, control prenatal"))
print(cuit_anonymizer.anonymize("Paciente c.u.i.t 25273842344 control prenatal"))
print(cuit_anonymizer.anonymize("Refiere extravio cuit sin sintomatologia tel 27384234"))

Paciente CUIT  XNUM_CUIT_CUILX control prenatal semana 27
Paciente cuil  XNUM_CUIT_CUILX, control prenatal
Paciente c.u.i.t  XNUM_CUIT_CUILX control prenatal
Refiere extravio cuit sin sintomatologia tel 27384234


In [21]:
documents['anonymized'] = documents['anonymized'].apply(cuit_anonymizer.anonymize)

In [22]:
documents[documents['anonymized'].str.contains('XNUM_CUIT_CUILX',case=False)]

,original,preprocessed,anonymized


### PASAPORTE

In [23]:
## Expresión regular para Pasaporte

regexp_passport_prefix = r'pasaporte(\sn\s?\.?\°?)?\s*[:\.]?\s*'
# Between 3 and 10 alphanumeric characters with at least one number
regexp_passport_numbers = r'\b[a-zA-Z0-9]*[0-9][a-zA-Z0-9]*\b'

passport_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_passport_prefix,
    content_regex=regexp_passport_numbers,
    re_flags=re.IGNORECASE,
    category='XPASAPORTEX',
    output_type=OUTPUT_TYPE,
)

print(passport_anonymizer.anonymize("Paciente pasaporte AAA342034, control prenatal semana 27"))
print(passport_anonymizer.anonymize("Paciente pasaporte n AAA342034, control prenatal semana 27"))
print(passport_anonymizer.anonymize("Paciente pasaporte n°: AAA342034, control prenatal semana 27"))
print(passport_anonymizer.anonymize("Paciente pasaporte 2527384234"))
print(passport_anonymizer.anonymize("Refiere extravio pasaporte sin sintomatologia tel 27384234"))

Paciente pasaporte  XPASAPORTEX, control prenatal semana 27
Paciente pasaporte n  XPASAPORTEX, control prenatal semana 27
Paciente pasaporte n°:  XPASAPORTEX, control prenatal semana 27
Paciente pasaporte  XPASAPORTEX
Refiere extravio pasaporte sin sintomatologia tel 27384234


In [24]:
documents['anonymized'] = documents['anonymized'].apply(passport_anonymizer.anonymize)

In [25]:
documents[documents['anonymized'].str.contains('XPASAPORTEX',case=False)]

,original,preprocessed,anonymized


### MATRICULA

In [26]:
## Expresión regular para MATRICULA
regexp_license_prefix = r'\bm(\.?)(p|n)(\.?)\s*[:\.]?\s*'
regexp_license_numbers = r'\b[0-9]{3,}\b'

license_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_license_prefix,
    content_regex=regexp_license_numbers,
    re_flags=re.IGNORECASE,
    category='XMATRICULAX',
    output_type=OUTPUT_TYPE,
)

print(license_anonymizer.anonymize("Atendido por dr. Ortiz mp 23411, control prenatal semana 27"))
print(license_anonymizer.anonymize("Atendido por dr. Ortiz m.p. 23411 control prenatal semana 27"))
print(license_anonymizer.anonymize("Atendido por dr. Ortiz m.n.: 23411 control prenatal semana 27"))
print(license_anonymizer.anonymize("Refiere extravio m.p. sin sintomatologia tel 27384234"))

Atendido por dr. Ortiz mp  XMATRICULAX, control prenatal semana 27
Atendido por dr. Ortiz m.p.  XMATRICULAX control prenatal semana 27
Atendido por dr. Ortiz m.n.:  XMATRICULAX control prenatal semana 27
Refiere extravio m.p. sin sintomatologia tel 27384234


In [27]:
documents['anonymized'] = documents['anonymized'].apply(license_anonymizer.anonymize)

In [28]:
documents[documents['anonymized'].str.contains('XMATRICULAX',case=False)]

,original,preprocessed,anonymized


### PATENTE

In [29]:
## Expresión regular para PATENTE
regexp_license_plate = r'\b([aA-zZ]{3}[0-9]{3})|([aA-zZ]{2}[0-9]{3}[aA-zZ]{2})\b'

license_plate_anonymizer = RegexAnnonimizer(
    content_regex=regexp_license_plate,
    category='XPATENTEX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(license_plate_anonymizer.anonymize("Auto patente AA345QP fuera de servicio"))
print(license_plate_anonymizer.anonymize("Auto patente aub325- fuera de servicio"))

Auto patente XPATENTEX fuera de servicio
Auto patente XPATENTEX- fuera de servicio


In [30]:
documents['anonymized'] = documents['anonymized'].apply(license_plate_anonymizer.anonymize)

In [31]:
documents[documents['anonymized'].str.contains('XPATENTEX', case=False)]

,original,preprocessed,anonymized
129,"Datos del paciente.\nNombre: Natalia Padilla De la Fuente\nDNI: 93306180X\nFecha de nacimiento: 19/02/1990\nGénero: Femenino\nDomicilio: Calle de Fuencarral 29, 10B\nCiudad: Aranda de Duero, Orense, Galicia\nCódigo postal: 32229\nEmail: natalia-padilla_delafuente@aol.com\nTeléfono fijo: +34 988 92 34 73\nTeléfono móvil: +34 788 64 25 84\nNHC: 5052916\nNASS: 097272229925\nCondición de riesgo: Operador de Maquinaria Pesada\nDatos asistenciales.\nMédico: Dra. María Asunción Crespo Guzman. NC 838560104. Especialista en Enfermedades Infecciosas. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 01/09/2018\nEpisodio: 61727998\nCentro de salud: Centro de Salud San Blas\nMatrícula del coche: 3650OVO\nModelo: Mazda CX-5\nVIN: VSQ368DR70V377478\nInforme clínico del paciente:\nPaciente vegano de 28 años de edad, acompañado de su hija, con antecedentes de alergia a los productos lácteos y carne. Presenta un cuadro de gastroenteritis aguda, con síntomas de diarrea, vómitos y dolor abdominal. La exploración física revela: temperatura 38,5°C, presión arterial 120/80 mmHg, frecuencia cardíaca 100 lpm. Se encuentra consciente, orientado, con buen estado de nutrición e hidratación. La exploración abdominal muestra signos de irritación intestinal, sin signos de obstrucción o perforación. Los análisis de sangre muestran: hemoglobina 12,5 g/dl, leucocitos 15.000/mm3, creatinina 1,2 mg/dl. Se solicitan pruebas de infección gastrointestinal, incluyendo cultivos de heces y sangre. El paciente es tratado con líquidos intravenosos y medicamentos antieméticos. Se le recomienda una dieta blanda y se le prescribe un curso de antibióticos en caso de que los análisis de sangre confirmen una infección.\nEvolución y comentarios:\nEl paciente muestra una respuesta favorable al tratamiento, con disminución de los síntomas y mejora en el estado general. Se le realiza una exploración abdominal con ultrasonido, que muestra signos de inflamación intestinal sin signos de obstrucción o perforación. Se le prescribe un curso de probióticos para ayudar a restaurar la flora intestinal.\nRemitido por:\nDra. María Asunción Crespo Guzman, Especialista en Enfermedades Infecciosas, Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC).\n","Datos del paciente.\nNombre: Natalia Padilla De la Fuente\nDNI: 93306180X\nFecha de nacimiento: 19/02/1990\nGenero: Femenino\nDomicilio: Calle de Fuencarral 29, 10B\nCiudad: Aranda de Duero, Orense, Galicia\nCodigo postal: 32229\nEmail: natalia-padilla_delafuente@aol.com\nTelefono fijo: +34 988 92 34 73\nTelefono movil: +34 788 64 25 84\nNHC: 5052916\nNASS: 097272229925\nCondicion de riesgo: Operador de Maquinaria Pesada\nDatos asistenciales.\nMedico: Dra. Maria Asuncion Crespo Guzman. NC 838560104. Especialista en Enfermedades Infecciosas. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 01/09/2018\nEpisodio: 61727998\nCentro de salud: Centro de Salud San Blas\nMatricula del coche: 3650OVO\nModelo: Mazda CX-5\nVIN: VSQ368DR70V377478\nInforme clinico del paciente:\nPaciente vegano de 28 años de edad, acompañado de su hija, con antecedentes de alergia a los productos lacteos y carne. Presenta un cuadro de gastroenteritis aguda, con sintomas de diarrea, vomitos y dolor abdominal. La exploracion fisica revela: temperatura 38,5°C, presion arterial 120/80 mmHg, frecuencia cardiaca 100 lpm. Se encuentra consciente, orientado, con buen estado de nutricion e hidratacion. La exploracion abdominal muestra signos de irritacion intestinal, sin signos de obstruccion o perforacion. Los analisis de sangre muestran: hemoglobina 12,5 g/dl, leucocitos 15.000/mm3, creatinina 1,2 mg/dl. Se solicitan pruebas de infeccion gastrointestinal, incluyendo cultivos de heces y sangre. 

### OTROS_NUM

#### Historias clínicas

In [32]:
## Expresión regular para HISTORIA CLINICA
regexp_hc_prefix = r'\bh(\.?)c(\.?)\s*[:\.]?\s*'
regexp_hc_numbers = r'[0-9]{3,}'

clinic_history_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_hc_prefix,
    content_regex=regexp_hc_numbers,
    re_flags=re.IGNORECASE,
    category='XOTROS_NUMX',
    output_type=OUTPUT_TYPE,
)

print(clinic_history_anonymizer.anonymize("Atendido previamente hc 23411, control prenatal semana 27"))
print(clinic_history_anonymizer.anonymize("Atendido previamente h.c. 23411, control prenatal semana 27"))
print(clinic_history_anonymizer.anonymize("Atendido  previamente hc h.c.: 23411 control prenatal semana 27"))
print(clinic_history_anonymizer.anonymize("Refiere extravio hc sin sintomatologia tel 27384234"))

Atendido previamente hc  XOTROS_NUMX, control prenatal semana 27
Atendido previamente h.c.  XOTROS_NUMX, control prenatal semana 27
Atendido  previamente hc h.c.:  XOTROS_NUMX control prenatal semana 27
Refiere extravio hc sin sintomatologia tel 27384234


In [33]:
documents['anonymized'] = documents['anonymized'].apply(clinic_history_anonymizer.anonymize)

In [34]:
documents[documents['anonymized'].str.contains('XOTROS_NUMX', case=False)]

,original,preprocessed,anonymized


### DIRECCIÓN

In [35]:
streets = pd.read_csv("datasets/diccionario_calles_editadas.csv", sep=';', index_col=None)
streets.head(5)

,categoria,nombre,provincia_nombre,Token0,Token1,Token2,Token3,Token4,Token5,Token6
0,PJE,pje estudiante,La Rioja,pje,estudiante,NaN,NaN,NaN,NaN,NaN
1,CALLE,20 de mayo,La Rioja,20,de,mayo,NaN,NaN,NaN,NaN
2,CALLE,1 de enero,La Rioja,1,de,enero,NaN,NaN,NaN,NaN
3,CALLE,10 de julio,La Rioja,10,de,julio,NaN,NaN,NaN,NaN
4,CALLE,aconcagua,La Rioja,aconcagua,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Sanitize street names
street_names = [remove_accents(name).strip()
                for name in streets.nombre
                if not re.search(ILLEGAL_CHAR_REGEX, name)]
regexp_address_prefix = r'\b(calle|avenida|av\.?|pasaje|pje\.?|parcela|manzana|tira|bloque|casa|lote|barrio)\s*[:]?\s*'
# Nombre de calles o barrios incluyendo cualquier numero posterior
regexp_streets = '(' + '|'.join(street_names) + ')(\s+([0-9]{1,6}))?'

address_w_names_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_address_prefix,
    content_regex=regexp_streets,
    re_flags=re.IGNORECASE,
    category='XDIRECCIONX',
    output_type=OUTPUT_TYPE,
)

# Ejemplos
print(address_w_names_anonymizer.anonymize("Vive en av. antartida argentina 1470, se indica medicacion"))
print(address_w_names_anonymizer.anonymize("Direccion: calle: 10 de julio, Barrio: alfonsina storini con hermanos"))
print(address_w_names_anonymizer.anonymize("Vive en Barrio barrioquenoexiste con hermanos"))
print(address_w_names_anonymizer.anonymize("Vive en manzana 3 lote 9 barrio 10 de julio."))

Vive en av.  XDIRECCIONX, se indica medicacion
Direccion: calle:  XDIRECCIONX, Barrio:  XDIRECCIONX con hermanos
Vive en Barrio barrioquenoexiste con hermanos
Vive en manzana 3 lote 9 barrio  XDIRECCIONX.


<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_71236/2484243500.py:7: SyntaxWarning: invalid escape sequence '\s'
  regexp_streets = '(' + '|'.join(street_names) + ')(\s+([0-9]{1,6}))?'


In [37]:
regexp_address_prefix = r'\b(calle|avenida|pasaje|pje\.?|parcela|manzana|tira|bloque|casa|lote|barrio)\s*[:]?\s*'
# Palabra o numero
regexp_address_content = r'\b([0-9]{1,6}|[a-zA-Z]+)\b'

address_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_address_prefix,
    content_regex=regexp_address_content,
    re_flags=re.IGNORECASE,
    category='XDIRECCIONX',
    output_type=OUTPUT_TYPE,
)

# Ejemplos
print(address_anonymizer.anonymize("Vive en avenida antartida argentina 1470, se indica medicacion"))
print(address_anonymizer.anonymize("Direccion: calle: 10 de julio, Barrio: alfonsina storini con hermanos"))
print(address_anonymizer.anonymize("Vive en Barrio barrioquenoexiste con hermanos"))
print(address_anonymizer.anonymize("Vive en manzana 3 lote 9 barrio 10 de julio."))

Vive en avenida  XDIRECCIONX argentina 1470, se indica medicacion
Direccion: calle:  XDIRECCIONX de julio, Barrio:  XDIRECCIONX storini con hermanos
Vive en Barrio  XDIRECCIONX con hermanos
Vive en manzana  XDIRECCIONX lote  XDIRECCIONX barrio  XDIRECCIONX de julio.


In [38]:
documents['anonymized'] = documents['anonymized'].apply(address_w_names_anonymizer.anonymize)
documents['anonymized'] = documents['anonymized'].apply(address_anonymizer.anonymize)

In [39]:
documents[documents['anonymized'].str.contains('XDIRECCIONX', case=False)]

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: Emilia García López\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGénero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCódigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTeléfono fijo: +34 91 123 45 67\nTeléfono móvil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondición de riesgo: Persona con discapacidad\nDatos asistenciales.\nMédico: Dra. Sofía Pérez García. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clínico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con síntomas de dolor torácico radiante al brazo izquierdo y dificultad para respirar. La exploración física revela taquicardia, hipertensión y roncus bilaterales. Los análisis de sangre muestran elevación de la creatinina y la troponina. Se solicitan estudios de imagen (Rx tórax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventrículo izquierdo. Se instaura tratamiento con anticoagulantes y analgésicos, y se realiza una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertensión\n- Diabetes mellitus tipo 2\nEvolución y comentarios:\nEl paciente se encuentra en una evolución estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado análisis de sangre y estudios de imagen para confirmar el diagnóstico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgésicos. Se le realizará una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada.\nRemitido por:\nSe reenvía a la unidad de cardiología para seguir el tratamiento y la evolución del paciente.\n","Datos del paciente.\nNombre: Emilia Garcia Lopez\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGenero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCodigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTelefono fijo: +34 91 123 45 67\nTelefono movil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondicion de riesgo: Persona con discapacidad\nDatos asistenciales.\nMedico: Dra. Sofia Perez Garcia. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clinico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con sintomas de dolor toracico radiante al brazo izquierdo y dificultad para respirar. La exploracion fisica revela taquicardia, hipertension y roncus bilaterales. Los analisis de sangre muestran elevacion de la creatinina y la troponina. Se solicitan estudios de imagen (Rx torax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventriculo izquierdo. Se instaura tratamiento con anticoagulantes y analgesicos, y se realiza una angioplastia coronaria para restaurar la circulacion sanguinea en el area afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertension\n- Diabetes mellitus tipo 2\nEvolucion y comentarios:\nEl paciente se encuentra en una evolucion estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado analisis de sangre y estudios de imagen para confirmar el diagnostico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgesicos. Se le realizara una angioplastia coronaria para

### EFECTOR

Se siguen dos estrategias
- CAPS y Hospitales por diccionario con trigger words
- Siglas más usuales (HMN, HMyN, etc)

#### CAPS

In [40]:
caps = pd.read_csv("datasets/diccionario_caps.csv", sep=';', index_col=None)
caps.head(2)

,Nombre,Token0,Token1,Token2,Token3,Token4,Token5,Token6
0,caps 20 de mayo,caps,20,de,mayo,NaN,NaN,NaN
1,caps agua blanca,caps,agua,blanca,NaN,NaN,NaN,NaN


In [41]:
# Sanitize caps names
caps_names = [remove_accents(name).replace('caps', '').strip()
                for name in caps.Nombre
                if not re.search(ILLEGAL_CHAR_REGEX, name)]

regexp_caps_prefix = r'(CAPS|CPS)\s*[:\.]?'
regexp_caps_names = r'\b(' + '|'.join(caps_names) + r')\b'

caps_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_caps_prefix,
    content_regex=regexp_caps_names,
    re_flags=re.IGNORECASE,
    category='XCAPSX',
    output_type=OUTPUT_TYPE,
)

print(caps_anonymizer.anonymize("Atendido previamente en caps 20 de mayo, se indica medicacion"))
print(caps_anonymizer.anonymize("Atendido previamente en caps: agua blanca"))
print(caps_anonymizer.anonymize("Atendido previamente en cps: Agua Blanca"))
print(caps_anonymizer.anonymize("Atendido previamente en caps noexiste"))

Atendido previamente en caps  XCAPSX, se indica medicacion
Atendido previamente en caps: XCAPSX
Atendido previamente en cps: XCAPSX
Atendido previamente en caps noexiste


In [42]:
documents['anonymized'] = documents['anonymized'].apply(caps_anonymizer.anonymize)

In [43]:
documents[documents['anonymized'].str.contains('XCAPSX', case=False)]

,original,preprocessed,anonymized


#### Hospitales

In [44]:
hospitals = pd.read_csv("datasets/diccionario_hospitales.csv", sep=';', index_col=None)
hospitals.head(2)

,Nombre,Token0,Token1,Token2,Token3,Token4,Token5,Token6,Token7,Token8,Token9,Token10
0,hospital de la madre y el niño (la rioja) - capital,hospital,de,la,madre,y,el,niño,(la,rioja),-,capital
1,hospital campanas,hospital,campanas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Sanitize hospital names
hospitals_names = [
    remove_accents(name).replace('hospital', '').split('-')[0].split('(')[0].strip()
    for name in hospitals.Nombre
]
hospitals_names = [
    name for name in hospitals_names
    if not re.search(ILLEGAL_CHAR_REGEX, name)
]

regexp_hospital_prefix = r'Hospital|Hosp\.?|H(\.)?\s*[:\.]?'
regexp_hospital_names = r'\b(' + '|'.join(hospitals_names) + r')\b'
regexp_hospital_names = regexp_hospital_names.replace('.', '\.?')

hospital_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_hospital_prefix,
    content_regex=regexp_hospital_names,
    re_flags=re.IGNORECASE,
    category='XHOSPX',
    output_type=OUTPUT_TYPE,
)

print(hospital_anonymizer.anonymize("Atendido previamente en hospital segundo b. romero, se indica medicacion"))
print(hospital_anonymizer.anonymize("Atendido previamente en hospital segundo b romero, se indica medicacion"))
print(hospital_anonymizer.anonymize("Atendido previamente en hosp villa castelli"))
print(hospital_anonymizer.anonymize("Atendido previamente en H  Villa Castelli"))
print(hospital_anonymizer.anonymize("Atendido previamente en hosp por la maniana"))

Atendido previamente en hospital XHOSPX, se indica medicacion
Atendido previamente en hospital XHOSPX, se indica medicacion
Atendido previamente en hosp XHOSPX
Atendido previamente en H   XHOSPX
Atendido previamente en hosp por la maniana


<>:13: SyntaxWarning: invalid escape sequence '\.'
<>:13: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_71236/1765221295.py:13: SyntaxWarning: invalid escape sequence '\.'
  regexp_hospital_names = regexp_hospital_names.replace('.', '\.?')


In [46]:
documents['anonymized'] = documents['anonymized'].apply(hospital_anonymizer.anonymize)

In [47]:
documents[documents['anonymized'].str.contains('XHOSPX', case=False)]

,original,preprocessed,anonymized


### INSTITUCIÓN

In [48]:
schools = pd.read_csv("datasets/diccionario_est_educativos.csv", sep=';', index_col=None)
schools.head(2)

,Nombre,Tipo,Numeración,Nombre_limpio,Token0,Token1,Token2,Token3,Token4,Token5,Token6,Token7,Token8,Token9,Token10,Token11,Token12,Token13,Token14
0,esc.n°230 bajo hondo,esc.,n°230,bajo hondo,bajo,hondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,esc.n°353 batallon de ing. de construcciones 141,esc.,n°353,batallon de ing. de construcciones 141,batallon,de,ing.,de,construcciones,141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Sanitize school names
# Remove content in ()
school_names = [
    re.sub(r'\(.*?\)', '', name).replace('(', '').replace(')', '').strip()
    for name in schools.Nombre_limpio if isinstance(name, str)
]
school_names = set([
    remove_accents(name)
    for name in school_names if len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
])

regexp_school_main = r'escuela|esc\.|esc|colegio|jardin|infantes|instituto'
regexp_school_detail = r'primari[ao]|normal|secundari[ao]|privad[ao]|maternal|especial'
regexp_school_number = r'(\sn\s?\.?\°?)?\s*[0-9]+'
regexp_school_prefix = rf'({regexp_school_main})(?:\s+(?:{regexp_school_detail}))?'

regexp_school_names = '|'.join(school_names)
regexp_school_names = regexp_school_names.replace('.', '\.?')
regexp_school_names = rf'({regexp_school_number}\s?)?({regexp_school_names})'

# Anonimiza el nombre de los establecimientos
school_name_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_school_prefix,
    content_regex=regexp_school_names,
    re_flags=re.IGNORECASE,
    category='XINSTITUCIONX',
    output_type=OUTPUT_TYPE,
)

print(school_name_anonymizer.anonymize("Asiste a jardin maternal rincon de luz."))
print(school_name_anonymizer.anonymize("Asiste a colegio bajo hondo."))
print(school_name_anonymizer.anonymize("Asiste a colegio secundario bajo hondo."))
print(school_name_anonymizer.anonymize("Asiste a esc normal dr. jose santos salinas por la tarde"))
print(school_name_anonymizer.anonymize("Asiste a esc normal inexistente"))
print(school_name_anonymizer.anonymize("Asiste a esc n 345 munic.beatriz de la v.de costa medina"))
print(school_name_anonymizer.anonymize("Asiste a Esc. n° 345 para el paicor"))

<>:18: SyntaxWarning: invalid escape sequence '\.'
<>:18: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_71236/1147442673.py:18: SyntaxWarning: invalid escape sequence '\.'
  regexp_school_names = regexp_school_names.replace('.', '\.?')


Asiste a jardin maternal XINSTITUCIONX.
Asiste a colegio XINSTITUCIONX.
Asiste a colegio secundario XINSTITUCIONX.
Asiste a esc XINSTITUCIONX por la tarde
Asiste a esc normal inexistente
Asiste a esc XINSTITUCIONX
Asiste a Esc. n° 345 para el paicor


In [50]:
# Anonimiza solo el numero
school_number_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_school_prefix,
    content_regex=regexp_school_number,
    re_flags=re.IGNORECASE,
    category='XINSTITUCIONX',
    output_type=OUTPUT_TYPE,
)

print(school_number_anonymizer.anonymize("Asiste a esc normal inexistente"))
print(school_number_anonymizer.anonymize("Asiste a esc n 345 munic.beatriz de la v.de costa medina"))
print(school_number_anonymizer.anonymize("Asiste a Esc. n° 345 para el paicor"))

Asiste a esc normal inexistente
Asiste a esc XINSTITUCIONX munic.beatriz de la v.de costa medina
Asiste a Esc. XINSTITUCIONX para el paicor


In [51]:
documents['anonymized'] = documents['anonymized'].apply(school_name_anonymizer.anonymize)
documents['anonymized'] = documents['anonymized'].apply(school_number_anonymizer.anonymize)

In [52]:
documents[documents['anonymized'].str.contains('XINSTITUCIONX', case=False)]

,original,preprocessed,anonymized


#### Grados

In [53]:
# regexp_grado = re.compile(r'^grado$', re.IGNORECASE)
# regexp_grado_letras = re.compile(r'^primer$|^segundo$|^tercer$|^cuarto$|^quinto$|^sexto$|^septimo$', re.IGNORECASE)
# regexp_grado_combinado = re.compile(r'[0-9]{1}[a-z]{1,3}|[0-9]{1}º', re.IGNORECASE)
# regexp_grado_exclusión = re.compile(r'obesidad|quemadura|evoluci[oóò]n', re.IGNORECASE)

# def deidentificar_grado(texto_a_deidentificar):
#     lista_texto_a_deidentificar=texto_a_deidentificar.split(' ')
    
#     ## Iteración entre tokens de la lista del texto a deidentificar
#     for j, token in enumerate(lista_texto_a_deidentificar):
    
#         # Detecta si el token actual es grado
#         if regexp_grado.search(lista_texto_a_deidentificar[j]):
#             #Excluyo grados de obesidad y quemaduras
#             if not regexp_grado_exclusión.search(lista_texto_a_deidentificar[j-3]) or regexp_grado_exclusión.search(lista_texto_a_deidentificar[j-2]):
#                 if regexp_grado_letras.search(lista_texto_a_deidentificar[j-1]) or regexp_grado_combinado.search(lista_texto_a_deidentificar[j-1]):
#                 # Reemplazar el token actual por el patrón correspondiente
#                     if output_tipo==1:
#                         lista_texto_a_deidentificar[j] = '<'+ lista_texto_a_deidentificar[j] +'>'
#                         lista_texto_a_deidentificar[j-1] = '<'+ lista_texto_a_deidentificar[j-1] +'>'
#                     if output_tipo==2:
#                         lista_texto_a_deidentificar[j] = 'XXXX'
#                         lista_texto_a_deidentificar[j-1] = 'XXXX'
#                     if output_tipo==3:
#                         lista_texto_a_deidentificar[j] = 'XGRADOX'
#                         lista_texto_a_deidentificar[j-1] = 'XGRADOX'
    
#     # Generar el texto deidentificado concatenando todos los tokens ya revisados (y modificados si corrrespondiera)
#     return ' '.join(lista_texto_a_deidentificar)
    
# textos_a_deidentificar['texto_anonimizado'] = textos_a_deidentificar['texto_anonimizado'].apply(deidentificar_grado)

#### Turnos

In [54]:
## Expresión regular para TURNOS
regexp_shcool_turn = r'\b(turno mañana|turno tarde|TT|TM)\b'

shcool_turn_anonymizer = RegexAnnonimizer(
    content_regex=regexp_shcool_turn,
    category='XTURNOX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(shcool_turn_anonymizer.anonymize("Asiste a esc en turno tarde"))
print(shcool_turn_anonymizer.anonymize("Asiste a esc sexto grado TT por la mañana,"))
print(shcool_turn_anonymizer.anonymize("Asiste a esc sexto grado turno mañana."))

Asiste a esc en XTURNOX
Asiste a esc sexto grado XTURNOX por la mañana,
Asiste a esc sexto grado XTURNOX.


In [55]:
documents['anonymized'] = documents['anonymized'].apply(school_name_anonymizer.anonymize)

In [56]:
documents[documents['anonymized'].str.contains('XTURNOX', case=False)]

,original,preprocessed,anonymized


### ZONA

In [57]:
neighborhood = pd.read_csv("datasets/diccionario_barrios_populares.csv", sep=';')
province = pd.read_csv("datasets/diccionario_provincias.csv", sep=';')
city = pd.read_csv("datasets/diccionario_localidades.csv", sep=';')
county = pd.read_csv("datasets/diccionario_municipios.csv", sep=';')

county.head()

,ciudad,Token0,Token1,Token2,Token3,Token4
0,aimogasta,aimogasta,NaN,NaN,NaN,NaN
1,arauco,arauco,NaN,NaN,NaN,NaN
2,aminga,aminga,NaN,NaN,NaN,NaN
3,castro barros,castro,barros,NaN,NaN,NaN
4,chamical,chamical,NaN,NaN,NaN,NaN


In [58]:
location_names = (
    neighborhood['nombre'].values.tolist() + county['ciudad'].values.tolist() +
    city['nombre'].values.tolist() + province['provincia_nombre'].values.tolist()
)

location_names = set([
    remove_accents(name)
    for name in location_names if len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
])

regexp_locations = r'\b(' + '|'.join(location_names) + r')\b'

location_anonymizer = RegexAnnonimizer(
    content_regex=regexp_locations,
    category='XZONAX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(location_anonymizer.anonymize("Paciente se traslada desde Chamical para consulta"))
print(location_anonymizer.anonymize("Paciente se traslada desde CASTRO BARROS, para consulta"))

Paciente se traslada desde XZONAX para consulta
Paciente se traslada desde XZONAX, para consulta


In [59]:
documents['anonymized'] = documents['anonymized'].apply(location_anonymizer.anonymize)

In [60]:
documents[documents['anonymized'].str.contains('XZONAX', case=False)]

,original,preprocessed,anonymized
1,"Datos del paciente.\nNombre: Iñigo Escobar Rivero\nDNI: 97790373P\nFecha de nacimiento: 08/09/1944\nGénero: Masculino\nDomicilio: Calle de los Embajadores 88\nCiudad: Telde, Gerona, Cataluña\nCódigo postal: 17012\nEmail: inigo-escobar@imibic.org\nTeléfono fijo: +34 972 65 06 71\nTeléfono móvil: +34 772 38 57 17\nNHC: 8245120\nNASS: 724651062765\nCondición de riesgo: Trabajador de Sanitización\nDatos asistenciales.\nMédico: Dra. Emma Vicente Santos. NC 588434040. Neuróloga Investigadora en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 24/02/1975\nEpisodio: 00580872\nCentro de salud: Centro de Salud Chamartín\nInforme clínico del paciente:\nPaciente migrante de 30 años de edad, originario de un país en vías de desarrollo, que acude al servicio de urgencias con síntomas de dolor de cabeza intensos y frecuencia urinaria. Se encuentra en buen estado general, con una temperatura de 36,5°C y presión arterial de 120/80 mmHg. La exploración física revela una tensión muscular en el cuello y una pequeña inflamación en el ojo izquierdo.\nAntecedentes:\nEl paciente informa que ha estado trabajando en un hospital durante los últimos 5 años, donde ha estado expuesto a diferentes tipos de agentes químicos y biológicos. También informa que ha tenido problemas de salud en el pasado, incluyendo una infección por VIH que fue tratada con éxito.\nEvolución y comentarios:\nEl paciente ha sido tratado con analgésicos y antiinflamatorios, y se le ha realizado una serie de pruebas para determinar la causa de sus síntomas. Los resultados de las pruebas han sido inconclusivos, y se le ha recomendado que se someta a una serie de pruebas adicionales para determinar la causa de sus síntomas.\nRemitido por:\nEl paciente ha sido remitido a un especialista en medicina interna para que se le realicen más pruebas y se le diagnostique y trate adecuadamente.\n","Datos del paciente.\nNombre: Iñigo Escobar Rivero\nDNI: 97790373P\nFecha de nacimiento: 08/09/1944\nGenero: Masculino\nDomicilio: Calle de los Embajadores 88\nCiudad: Telde, Gerona, Cataluña\nCodigo postal: 17012\nEmail: inigo-escobar@imibic.org\nTelefono fijo: +34 972 65 06 71\nTelefono movil: +34 772 38 57 17\nNHC: 8245120\nNASS: 724651062765\nCondicion de riesgo: Trabajador de Sanitizacion\nDatos asistenciales.\nMedico: Dra. Emma Vicente Santos. NC 588434040. Neurologa Investigadora en Trastornos Cognitivos. Instituto de Investigacion Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Cordoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 24/02/1975\nEpisodio: 00580872\nCentro de salud: Centro de Salud Chamartin\nInforme clinico del paciente:\nPaciente migrante de 30 años de edad, originario de un pais en vias de desarrollo, que acude al servicio de urgencias con sintomas de dolor de cabeza intensos y frecuencia urinaria. Se encuentra en buen estado general, con una temperatura de 36,5°C y presion arterial de 120/80 mmHg. La exploracion fisica revela una tension muscular en el cuello y una pequeña inflamacion en el ojo izquierdo.\nAntecedentes:\nEl paciente informa que ha estado trabajando en un hospital durante los ultimos 5 años, donde ha estado expuesto a diferentes tipos de agentes quimicos y biologicos. Tambien informa que ha tenido problemas de salud en el pasado, incluyendo una infeccion por VIH que fue tratada con exito.\nEvolucion y comentarios:\nEl paciente ha sido tratado con analgesicos y antiinflamatorios, y se le ha realizado una serie de pruebas para determinar la causa de sus sintomas. Los resultados de las pruebas han sido inconclusivos, y se le ha recomendado que se someta a una serie de pruebas adicionales para determinar la causa de sus sintomas.\nRemitido por:\nEl paciente ha sido remitido a un especialista en medicina interna para que se le realicen mas pruebas y se le diagnostique y trate adecuadamente.\n","Datos del pac

### PAÍS

Preparación del listado de paises a detectar:

In [61]:
countries = pd.read_csv("datasets/diccionario_paises.csv", sep=';', index_col=None)

# Filtro algunas entradas teniendo en cuenta que su uso como país es muy infrecuente en el nuestro.
p1 = re.compile(
    'Nueva|Reunion|San Martin|Islas Cocos \(Keeling\)|india|malta|montenegro|guadalupe|mauricio|laos|'
    'eritrea|iran|polonia|portugal|camboya|islas malvinas|antartida|argentina',
    re.IGNORECASE)

countries = countries[~countries['nombre'].str.contains(p1)]
countries.tail()

<>:5: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_71236/737399818.py:5: SyntaxWarning: invalid escape sequence '\('
  'Nueva|Reunion|San Martin|Islas Cocos \(Keeling\)|india|malta|montenegro|guadalupe|mauricio|laos|'


,nombre,Token0,Token1,Token2,Token3,Token4,Token5,Token6,Token7
243,wallis y futuna,wallis,y,futuna,NaN,NaN,NaN,NaN,NaN
244,yemen,yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,yibuti,yibuti,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,zambia,zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,zimbabue,zimbabue,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
country_names = set([
    remove_accents(name).strip()
    for name in countries.nombre if len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
])

regexp_countries = r'\b(' + r'|'.join(country_names) + r')\b'

country_anonymizer = RegexAnnonimizer(
    content_regex=regexp_countries,
    category='XPAISX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(country_anonymizer.anonymize("Paciente se traslada desde ZAMBIA para consulta"))
print(country_anonymizer.anonymize("Paciente se traslada desde wallis y futuna, para consulta"))

Paciente se traslada desde XPAISX para consulta
Paciente se traslada desde XPAISX, para consulta


In [63]:
documents['anonymized'] = documents['anonymized'].apply(country_anonymizer.anonymize)

In [64]:
documents[documents['anonymized'].str.contains('XPAISX', case=False)]

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: Emilia García López\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGénero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCódigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTeléfono fijo: +34 91 123 45 67\nTeléfono móvil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondición de riesgo: Persona con discapacidad\nDatos asistenciales.\nMédico: Dra. Sofía Pérez García. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clínico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con síntomas de dolor torácico radiante al brazo izquierdo y dificultad para respirar. La exploración física revela taquicardia, hipertensión y roncus bilaterales. Los análisis de sangre muestran elevación de la creatinina y la troponina. Se solicitan estudios de imagen (Rx tórax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventrículo izquierdo. Se instaura tratamiento con anticoagulantes y analgésicos, y se realiza una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertensión\n- Diabetes mellitus tipo 2\nEvolución y comentarios:\nEl paciente se encuentra en una evolución estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado análisis de sangre y estudios de imagen para confirmar el diagnóstico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgésicos. Se le realizará una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada.\nRemitido por:\nSe reenvía a la unidad de cardiología para seguir el tratamiento y la evolución del paciente.\n","Datos del paciente.\nNombre: Emilia Garcia Lopez\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGenero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCodigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTelefono fijo: +34 91 123 45 67\nTelefono movil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondicion de riesgo: Persona con discapacidad\nDatos asistenciales.\nMedico: Dra. Sofia Perez Garcia. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clinico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con sintomas de dolor toracico radiante al brazo izquierdo y dificultad para respirar. La exploracion fisica revela taquicardia, hipertension y roncus bilaterales. Los analisis de sangre muestran elevacion de la creatinina y la troponina. Se solicitan estudios de imagen (Rx torax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventriculo izquierdo. Se instaura tratamiento con anticoagulantes y analgesicos, y se realiza una angioplastia coronaria para restaurar la circulacion sanguinea en el area afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertension\n- Diabetes mellitus tipo 2\nEvolucion y comentarios:\nEl paciente se encuentra en una evolucion estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado analisis de sangre y estudios de imagen para confirmar el diagnostico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgesicos. Se le realizara una angioplastia coronaria para

### PERSONA

Chequea token a token si:

- ~~comienza con mayúscual y sigue con minúsculas~~
- es un nombre,
- es un apellido

In [65]:
## Lista de nombres en minúscula, sin números, con al menos 3 letras, sin stop Words y con frecuencia mayor a 5000.
first_names = pd.read_csv("datasets/nombres_editados.csv", sep=';', index_col = None)['nombre_limpio'].to_list()
last_names = pd.read_csv("datasets/apellidos_unicos.csv", sep=';', index_col = None)['apellido'].to_list()
names = [
    remove_accents(name).strip()
    for name in first_names + last_names
    if isinstance(name, str) and len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
]
names[:5]

['aaron', 'abel', 'abelardo', 'abigail', 'abraham']

In [66]:
len(set(names))

75911

In [67]:
# Exclude common noun names from list
noun_names = [
    'papa', 'tamaño', 'salud', 'hospital', 'niño', 'medico', 'toma', 'fecha', 'rodilla', 'vigil',
    'abajo', 'alegre', 'abusar', 'cola', 'cancela', 'grande', 'cortes', 'saliva', 'sola', 
    'areas', 'bajo', 'bien', 'blando', 'bota', 'buena', 'pareja', 'claro', 'claros', 'colica', 
    'cornea', 'cristalino', 'agua', 'corto', 'tiempo', 'grado', 'dieta', 'entrada', 'esposo', 
    'izquierdo', 'genera', 'general', 'genero', 'grados', 'haber', 'hierro', 'tres', 'igual', 
    'indico', 'plano', 'ligero', 'mañana', 'medica', 'medio', 'mucha', 'nacer', 'noto', 
    'parejas', 'pares', 'parto', 'pene', 'peso', 'piernas', 'requiere', 'semana', 'vaginal', 
    'varon', 'vino', 'nacimiento', 'hitos', 'menor', 'cordon', 'cuello', 'guardia', 'muñeca',
    'factor', 'chagas', 'sano', 'camino', 'manera', 'signo', 'come', 'solo', 'sola', 'cera',
    'tapon', 'cabeza', 'base', 'lenta', 'patron', 'borde', 'abuela', 'alli', 'alto', 'amigo',
    'anillo', 'aporta', 'area', 'arriba', 'bajar', 'balon', 'barrio', 'beber', 'boca',
    'brindo', 'bueno', 'busca', 'calle', 'calores', 'campo', 'cara', 'cargo', 'casa',
    'casi', 'caso', 'causa', 'cayo', 'cece', 'cena', 'chica', 'chico', 'constante',
    'cuadro', 'cuesta', 'debi', 'debia', 'deseo', 'deseo', 'DESEO', 'dias', 'dios',
    'done', 'dura', 'duro', 'escalera', 'estudia', 'figura', 'foco', 'Fondo', 'fosas',
    'fuertes', 'gato', 'gotas', 'guarda', 'hacen', 'haga', 'Jardin', 'labo', 'lado', 'lava',
    'legal', 'ligo', 'mala', 'mando', 'mano', 'mares', 'matar', 'mayor', 'medicina',
    'mesas', 'mori', 'mover', 'nace', 'nota', 'nuevo', 'pala', 'panico', 'pared', 'paro',
    'placer', 'presentado', 'presta', 'primo', 'publico', 'quiero', 'quizas', 'raiz',
    'redes', 'respeto', 'riesgo', 'rojo', 'rota', 'ruta', 'saber', 'sabia', 'sale',
    'sangra', 'segundo', 'segura', 'seis', 'severo', 'sobrino', 'soma', 'sule', 'teme',
    'terceros', 'tira', 'tirado', 'torno', 'trabajo', 'tranquilo', 'valor', 'vaya',
    'vender', 'vida', 'vital', 'vive', 'vivian', 'vuelo', 'comer', 'fuerte', 'doble', 'sorda',
    'numero', 'dolores', 'rostro', 'paterno', 'colon', 'diab', 'corte', 'crema',
    'bazo', 'camara', 'rabia', 'fuma', 'calcio', 'indice', 'timpano', 'mandato', 'escuela',
    'venia', 'vecino', 'terreno', 'sordo', 'salia', 'sala', 'saco', 'regalo', 'reales',
    'real', 'rato', 'rama', 'puerta', 'plata', 'pase', 'molares', 'llano', 'feto',
    'espinal', 'decia', 'cosa', 'carta', 'carne', 'cabello', 'brusco', 'bordes', 'bases',
    'baño', 'baba', 'avanza', 'amor', 'labio', 'maligno', 'teta'
]

names = set([name for name in names if name not in noun_names])
len(set(names))

75654

In [68]:
regexp_names = r'\b({})\b'.format('|'.join(names))

name_anonymizer = RegexAnnonimizer(
    content_regex=regexp_names,
    category='XPERSONAX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
    verbose=True
)

print(name_anonymizer.anonymize("Paciente con tumor de gran tamaño medico con ibuprofeno."))
print(name_anonymizer.anonymize("Caida del cordon umbilical."))
print(name_anonymizer.anonymize("Acude a consulta con hermano Abel."))

Paciente con tumor de gran tamaño medico con ibuprofeno.
Caida del cordon umbilical.
Replacing Abel
Acude a consulta con hermano XPERSONAX.


In [69]:
documents['anonymized'] = documents['anonymized'].apply(name_anonymizer.anonymize)

Replacing Emilia
Replacing Garcia
Replacing Lopez
Replacing Ciudad
Replacing Madrid
Replacing NASS
Replacing Sofia
Replacing Perez
Replacing Garcia
Replacing Paz
Replacing Castellana
Replacing Madrid
Replacing Villaverde
Replacing Iñigo
Replacing Escobar
Replacing Rivero
Replacing Ciudad
Replacing NASS
Replacing Emma
Replacing Vicente
Replacing Santos
Replacing Madrid
Replacing pais
Replacing trate
Replacing Manuela
Replacing Morales
Replacing Cabrera
Replacing Ciudad
Replacing Elda
Replacing Teruel
Replacing Aragon
Replacing NASS
Replacing Susana
Replacing Carmona
Replacing Pidal
Replacing Pidal
Replacing bocio
Replacing barriga
Replacing amarillo
Replacing Juan
Replacing Perez
Replacing Garcia
Replacing Alejandro
Replacing Galvez
Replacing Luna
Replacing Ciudad
Replacing Palma
Replacing Islas
Replacing NASS
Replacing Omar
Replacing Guerrero
Replacing Maldonado
Replacing Lemos
Replacing Madrid
Replacing Eric
Replacing Gomez
Replacing Ciudad
Replacing NASS
Replacing Pintor
Replacing Es

In [70]:
documents[documents['anonymized'].str.contains('XPERSONAX', case=False)]

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: Emilia García López\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGénero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCódigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTeléfono fijo: +34 91 123 45 67\nTeléfono móvil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondición de riesgo: Persona con discapacidad\nDatos asistenciales.\nMédico: Dra. Sofía Pérez García. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clínico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con síntomas de dolor torácico radiante al brazo izquierdo y dificultad para respirar. La exploración física revela taquicardia, hipertensión y roncus bilaterales. Los análisis de sangre muestran elevación de la creatinina y la troponina. Se solicitan estudios de imagen (Rx tórax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventrículo izquierdo. Se instaura tratamiento con anticoagulantes y analgésicos, y se realiza una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertensión\n- Diabetes mellitus tipo 2\nEvolución y comentarios:\nEl paciente se encuentra en una evolución estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado análisis de sangre y estudios de imagen para confirmar el diagnóstico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgésicos. Se le realizará una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada.\nRemitido por:\nSe reenvía a la unidad de cardiología para seguir el tratamiento y la evolución del paciente.\n","Datos del paciente.\nNombre: Emilia Garcia Lopez\nDNI: 12345678A\nFecha de nacimiento: 12/02/1968\nGenero: Mujer\nDomicilio: Calle de la Princesa 123\nCiudad: Madrid\nCodigo postal: 28008\nEmail: emilia.garcia@ucm.es\nTelefono fijo: +34 91 123 45 67\nTelefono movil: +34 66 789 01 23\nNHC: 1234567\nNASS: 987654321098\nCondicion de riesgo: Persona con discapacidad\nDatos asistenciales.\nMedico: Dra. Sofia Perez Garcia. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.\nFecha de ingreso: 20/03/2022\nCentro de salud: Centro de Salud Villaverde\nInforme clinico del paciente:\nPaciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con sintomas de dolor toracico radiante al brazo izquierdo y dificultad para respirar. La exploracion fisica revela taquicardia, hipertension y roncus bilaterales. Los analisis de sangre muestran elevacion de la creatinina y la troponina. Se solicitan estudios de imagen (Rx torax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventriculo izquierdo. Se instaura tratamiento con anticoagulantes y analgesicos, y se realiza una angioplastia coronaria para restaurar la circulacion sanguinea en el area afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.\nAntecedentes:\n- Enfermedad cardiovascular\n- Hipertension\n- Diabetes mellitus tipo 2\nEvolucion y comentarios:\nEl paciente se encuentra en una evolucion estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado analisis de sangre y estudios de imagen para confirmar el diagnostico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgesicos. Se le realizara una angioplastia coronaria para

### DRX

In [71]:
# ## Expresión regular para DRX

# # Distancia máxima tolerada entre el token trigger y el último que puede ser marcado como esa entidad.
# dist_token = 15

# # originales-------------------------------------------
# #titulos = '((?<=\.| )|^)dr(?=(a|\.| ))|doctor(a?)|((?<=\.| )|^)licenciad[a|o]|((?<=\.| )|^)lic(\.| ?)'
# #-------------------------------------------------------

# regexp_drx = re.compile(r'((?<=\.| )|^)dr(?=(a|\.| |$))|doctor(a?)|((?<=\.| )|^)licenciad[a|o]|((?<=\.| )|^)lic(\.| ?)',
#                         re.IGNORECASE)

# regexp_drx_mayusculas = re.compile(r'^[A-Z]{1}[a-z]+')

# def deidentificar_drx(texto_a_deidentificar):
#     # Genero la lista a partir del texto_preprocesado de esa fila
#     lista_texto_a_deidentificar = re.split('([\.,:\? -])',texto_a_deidentificar)
    
#     ## Iteración entre tokens de la lista del texto a deidentificar
#     drx_flag = False
#     for j,token in enumerate(lista_texto_a_deidentificar):
#         # A partir del segundo token
#         if j > 0:
#             # Si el token actual es precedido por dr/doc,etc
#             if regexp_drx.search(lista_texto_a_deidentificar[j-1]):
#                 drx_flag=True
#                 pos_token = j
#                 # Si el token actual comienza con mayúscula
# #                 if (lista_texto_a_deidentificar[j].lower() in lista_nombres) or regexp_drx_mayusculas.search(lista_texto_a_deidentificar[j]):
# #                 # Reemplazar el token actual por el patrón correspondiente
# #                     if output_tipo==1:
# #                         lista_texto_a_deidentificar[j] = '<'+ lista_texto_a_deidentificar[j] +'>'
# #                     if output_tipo==2:
# #                         lista_texto_a_deidentificar[j] = 'XXXX'
# #                     if output_tipo==3:
# #                         lista_texto_a_deidentificar[j] = '<DRX>'
#             # Si el token actual es un nombre o apellido
# #             if drx == True and (lista_texto_a_deidentificar[j].lower() in lista_nombres or regexp_drx_mayusculas.search(lista_texto_a_deidentificar[j])) and j-pos_token<dist_token:
#             if drx_flag == True and (lista_texto_a_deidentificar[j]=='XPERSONAX' and j-pos_token<dist_token):
#                     # Reemplazar el token actual por el patrón correspondiente
#                     if output_tipo==1:
#                         lista_texto_a_deidentificar[j] = '<'+ lista_texto_a_deidentificar[j] +'>'
#                     if output_tipo==2:
#                         lista_texto_a_deidentificar[j] = 'XXXX'
#                     if output_tipo==3:
#                         lista_texto_a_deidentificar[j] = 'XDRXX'    
# #             else:
# #                 drx_flag == False
    
#     # Generar el texto deidentificado concatenando todos los tokens ya revisados (y modificados si corrrespondiera)
#     return ''.join(lista_texto_a_deidentificar)
    
# textos_a_deidentificar['texto_anonimizado'] = textos_a_deidentificar['texto_anonimizado'].apply(deidentificar_drx)
# #     Aclarar por qué busco en listas (minus y mayus())

### EPOF

Enfermedades poco frecuentes

In [72]:
epof = pd.read_csv("./datasets/epof.csv", names=['enfermedad'], header=None)
lista_epof = epof['enfermedad'].to_list()
lista_epof[:10]

['Abetalipoproteinemia',
 'Abscesos asépticos sensibles a corticosteroides',
 'Acalasia - microcefalia',
 'Acalasia idiopática',
 'Acalvaria',
 'Acatalasemia',
 'Accidente cerebrovascular isquémico pediátrico',
 'Aceruloplasminemia',
 'Acidemia isovalérica',
 'Acidemia malónica y metilmalónica combinada']

In [73]:
epof_names = set([
    remove_accents(name).strip()
    for name in lista_epof if len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
])

regexp_epof = '|'.join(epof_names)

epof_anonymizer = RegexAnnonimizer(
    content_regex=regexp_epof,
    category='XEPoFX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(epof_anonymizer.anonymize("Paciente presenta acalvaria aguda."))
print(epof_anonymizer.anonymize("Paciente presenta Acidemia malonica y metilmalonica combinada."))

Paciente presenta XEPoFX aguda.
Paciente presenta XEPoFX.


In [74]:
documents['anonymized'] = documents['anonymized'].apply(epof_anonymizer.anonymize)

In [75]:
documents[documents['anonymized'].str.contains('XEPoFX', case=False)]

,original,preprocessed,anonymized
43,"Datos del paciente.\nNombre: Sebastián Espinosa Flores\nDNI: 75812855W\nFecha de nacimiento: 29/04/1969\nGénero: Hombre\nDomicilio: Calle de Arenal 3\nCiudad: Ciudad autónoma de Melilla, Melilla\nCódigo postal: 51940\nEmail: sebastian.espinosa-flores@ioba.med.uva.es\nTeléfono fijo: +34 951 17 35 28\nTeléfono móvil: +34 651 62 85 28\nNHC: 8846957\nNASS: 797769606499\nCondición de riesgo: Trabajador Industrial\nDatos asistenciales.\nMédico: Dra. Carla Román Marti. NC 694260634. Investigadora Principal en Parkinson. Instituto de Investigación Biomédica en Red de Enfermedades Neurodegenerativas (CIBERNED). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nCargo: Investigadora Principal\nDirección: Instituto de Investigación Biomédica en Red de Enfermedades Neurodegenerativas (CIBERNED). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 02/03/2008\nHospital: Complejo Asistencial Universitario de León\nInforme clínico del paciente:\nPaciente no fumador de 38 años de edad, acompañado de su madre, que acude al servicio de urgencias con síntomas de dolor abdominal crónico y pérdida de peso significativa en las últimas semanas. El paciente tiene antecedentes de hipertensión arterial y diabetes mellitus tipo 2, controladas con medicación.\nExploración física:\n- Tª: 36,5°C\n- TA: 120/80 mmHg\n- FC: 90 lpm\n- Conscientemente alterado, orientado, con sudoración y temperatura normal\n- Auscultación cardíaca sin anormalidades\n- Auscultación pulmonar normal\n- Abdomen: dolor en el hemicolo superior derecho, con signos de irritación y megalia\nExploración complementaria:\n- Hemograma: Hb 13,5 g/dl; leucocitos 12.000/mm3 (neutrófilos 70%)\n- Bioquímica: glucosa 120 mg/dl; urea 30 mg/dl; creatinina 1,2 mg/dl; sodio 140 mEq/l; potasio 3,8 mEq/l\n- Imagenología: exploración abdominal con ecografía revela un aumento de la velocidad de flujo en la vena porta y un aumento del índice de resistencia vascular en el hígado\nDiagnóstico:\n- Enfermedad hepática crónica, tipo C, con compromiso de la función hepática y de la circulación portal\nTratamiento:\n- Reposo absoluto\n- Antibióticos de amplio espectro\n- Medicación para controlar la función hepática y la circulación portal\n- Seguimiento estrecho en el servicio de gastroenterología\nEvolución y comentarios:\nEl paciente fue ingresado en la unidad de cuidados intensivos para estabilizar su estado clínico. Después de 48 horas de tratamiento, el paciente mostró una mejoría significativa en su estado clínico, con disminución de la temperatura y de la sudoración. El paciente fue trasladado a la unidad de gastroenterología para continuar su tratamiento y seguimiento.\n","Datos del paciente.\nNombre: Sebastian Espinosa Flores\nDNI: 75812855W\nFecha de nacimiento: 29/04/1969\nGenero: Hombre\nDomicilio: Calle de Arenal 3\nCiudad: Ciudad autonoma de Melilla, Melilla\nCodigo postal: 51940\nEmail: sebastian.espinosa-flores@ioba.med.uva.es\nTelefono fijo: +34 951 17 35 28\nTelefono movil: +34 651 62 85 28\nNHC: 8846957\nNASS: 797769606499\nCondicion de riesgo: Trabajador Industrial\nDatos asistenciales.\nMedico: Dra. Carla Roman Marti. NC 694260634. Investigadora Principal en Parkinson. Instituto de Investigacion Biomedica en Red de Enfermedades Neurodegenerativas (CIBERNED). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nCargo: Investigadora Principal\nDireccion: Instituto de Investigacion Biomedica en Red de Enfermedades Neurodegenerativas (CIBERNED). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 02/03/2008\nHospital: Complejo Asistencial Universitario de Leon\nInforme clinico del paciente:\nPaciente no fumador de 38 años de edad, acompañado de su madre, que acude al servicio de urgencias con sintomas de dolor abdominal cronico y perdida de peso significativa en las ultimas semanas. El paciente tiene antecedentes de hipertension arterial y diabetes mellitus tipo 2, controladas con medicacion.\nExploracion fisi

## 5. Devuelve los datos procesados (output)

### Transforma entidades

## 6. Guarda salida

In [76]:
documents.shape
output_filename =  '/home/usuario/Documentos/TrabajoEspecial/Modelos/REGEX/c) SPGExt/out.csv'
print('Saving to:', output_filename)
documents.to_csv(output_filename, sep=',', header=True, index=False)

Saving to: /home/usuario/Documentos/TrabajoEspecial/Modelos/REGEX/c) SPGExt/out.csv
